In [53]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
from datetime import datetime
pd.set_option('display.max_rows', 1000)
pd.options.display.max_colwidth = 1000

#to send emails
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#to make the conection with spreadsheets
import gspread
from google.oauth2.service_account import Credentials

# A. p_acquisition

### SET THE URLS IN VARIABLES

In [2]:
urls_ergonomia = [
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/tlv-myx-801-1/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn55bk/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cashoffice-silla-ergonomica/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cedric/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/noblewell/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/sihoo-lb14/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/ronda-silla-espanola/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/diablo-v-master/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/diablo-v-basic/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn61bkv1/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposapies/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn86bk/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/femor/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/",
"https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/umi/"   
]

In [3]:
#If want to add a new URL to the "ergonomia" category
def new_url_ergo(new_url):
    urls_ergonomia.append(new_url)
    return "new url 'ergonomia' added"

In [4]:
urls_oficina = [
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/songmics-obn52bk/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/songmics-obn22bk/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/allguest-cedric/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/hbada-hdny108bm-eu/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/intimate-wm-heart-sillon-b07x8tqh96/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/vinsetto-sillon-de-oficina-azul-claro/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/exofcer-mc6310/",
"https://sillasybienestar.com/oficina-y-escritorio/sillas-de-oficina/review-individual/songmics-obg24b/"
]

In [5]:
#If want to add a new URL to the "oficina" category
def new_url_oficina(new_url):
    urls_oficina.append(new_url)
    return "new url 'oficina' added"

In [6]:
urls_rodilla = [
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/duehome/",
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/himimi-silla-de-rodillas/",
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/varier/",
"https://sillasybienestar.com/ergonomia/sillas-de-rodillas/review-individual/cinius/"
]

In [7]:
#If want to add a new URL to the "rodilla" category
def new_url_rodilla(new_url):
    urls_rodilla.append(new_url)
    return "new url 'rodilla' added"

In [8]:
urls_gaming = [
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/gtplayer-rosa/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-gamer-2-0/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-silla-gamer-barata/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/nokaxus-yk-6008-rosa/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-nayuki/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/silla-gamer-bgeu-a136-sencillez-y-buen-precio-ofertas-2021/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/autofull-pink-bunny/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/adec-drw/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/femor-gaming/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/hbada-gaming-hdjy001bmj-cb/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/corsair-t3-rush/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/dxracer-king-ks06/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-horn/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-ray/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/dxracer-formula-f08/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-kitsune/",
"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-takamikura/"
#"https://sillasybienestar.com/gaming/sillas-gaming/review-individual/songmics-racing/" - no usamos
]

In [9]:
#If want to add a new URL to the "gaming" category
def new_url_gaming(new_url):
    urls_gaming.append(new_url)
    return "new url 'gaming' added"

In [10]:
#JOIN all the urls
def full_urls(urls_ergonomia, urls_oficina, urls_rodilla, urls_gaming):
    urls_products_list_list = urls_ergonomia + urls_oficina + urls_rodilla + urls_gaming
    return urls_products_list_list
    
urls_products_list = full_urls(urls_ergonomia, urls_oficina, urls_rodilla, urls_gaming)

### web scrapping

Obtain the HTML of all our URLs

In [11]:
def parsed_content(urls_products_list):
    #contenido "parseado" para las siguientes variables
    parsed_products_content_list = [bs4.BeautifulSoup(requests.get(i).content, "html.parser") for i in urls_products_list]
    return parsed_products_content_list

parsed_products_content_list = parsed_content(urls_products_list)

Obtain the price info of its class

In [12]:
def parsed_price_class(parsed_products_content_list):
    #contenido "parseado" para el precio
    parsed_products_price_class = [i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text for i in parsed_products_content_list]
    return parsed_products_price_class

parsed_products_price_class = parsed_price_class(parsed_products_content_list)

Obtain the final price

In [13]:
def product_price(parsed_products_content_list,urls_products_list):
    final_price_products_list =[]
    for i,e in zip(parsed_products_content_list, range(len(urls_products_list))):
        try:
            i.find_all("div",{"class":"wp-block-media-text__content"})[0]
            #si da error se ejecuta el except, sino el else
        except:
            if  urls_products_list[e] == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/":
                price_supuesto_0 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                price_supuesto_0_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_0)).replace(",",".")
                final_price_products_list.append(float(price_supuesto_0_cleaned))
                #final_price_products_status.append("correcto")
            else:
                final_price_products_list.append(-1)
                print("revisar funcion price, valor con -1")
                #final_price_products_status.append("revisar")
        else:
            if i.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip() == "No products found.":
                final_price_products_list.append(0)
                #final_price_products_status.append("descatalogado")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text == "":
                final_price_products_list.append(0)
                #final_price_products_status.append("sin_stock")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text != "":
                price_supuesto_1 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                price_supuesto_1_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_1)).replace(",",".")
                final_price_products_list.append(float(price_supuesto_1_cleaned))
                #final_price_products_status.append("correcto")
            else:
                print("revisar funcion price")
    return final_price_products_list

final_price_products_list = product_price(parsed_products_content_list,urls_products_list)

Obtain if the product is out of stock or discontinued

In [14]:
def product_status(parsed_products_content_list,urls_products_list):
    final_price_products_status =[]
    for i,e in zip(parsed_products_content_list, range(len(urls_products_list))):
        try:
            i.find_all("div",{"class":"wp-block-media-text__content"})[0]
            #si da error se ejecuta el except, sino el else
        except:
            if  urls_products_list[e] == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/mfavour/":
                #price_supuesto_0 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                #price_supuesto_0_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_0)).replace(",",".")
                #final_price_products_list.append(float(price_supuesto_0_cleaned))
                final_price_products_status.append("correcto")
            else:
                #final_price_products_list.append(-1)
                print("revisar funcion price, valor con -1")
                final_price_products_status.append("revisar")
        else:
            if i.find_all("div",{"class":"wp-block-media-text__content"})[0].text.strip() == "No products found.":
                #final_price_products_list.append(0)
                final_price_products_status.append("descatalogado")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text == "":
                #final_price_products_list.append(0)
                final_price_products_status.append("sin_stock")
            elif i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text != "":
                #price_supuesto_1 = i.find_all("span",{"class":"aawp-product__price aawp-product__price--current"})[0].text
                #price_supuesto_1_cleaned = re.sub("[^\d|\,]","",str(price_supuesto_1)).replace(",",".")
                #final_price_products_list.append(float(price_supuesto_1_cleaned))
                final_price_products_status.append("correcto")
            else:
                print("revisar funcion price")
    return final_price_products_status

final_price_products_status = product_status(parsed_products_content_list,urls_products_list)

Create a function to handle possible erros in the product information

In [15]:
def handling_error_vars_product(i,text):
    try:
        str(i).split(text)[1]
    except:
        return "none"
    else:
        return str(i).split(text)[1].split("';")[0].strip()

Obtain the NAME of the different products

In [16]:
def product_name(parsed_products_content_list):
    
    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
    
    final_name_products = [handling_error_vars_product(i,"ficha_product_name='") for i in parsed_products_content_list]
    return final_name_products

final_name_products = product_name(parsed_products_content_list)

Obtain the ID of the different products

In [17]:
def product_id(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_id_products = [handling_error_vars_product(i,"ficha_product_id='") for i in parsed_products_content_list]
    return final_id_products

final_id_products = product_id(parsed_products_content_list)

Obtain the BRAND of the different products

In [18]:
def product_brand(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
    
    final_brand_products = [handling_error_vars_product(i,"ficha_product_brand='") for i in parsed_products_content_list]
    return final_brand_products

final_brand_products = product_brand(parsed_products_content_list)

Obtain the DATE of the current day in different formats

1. With HYPHEN. Ex: 2022-04-14

In [19]:
def product_date_hyphen(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_date_hyphen_products = [datetime.today().strftime('%Y-%m-%d') for i in range(len(urls_products_list))]
    return final_date_hyphen_products

final_date_hyphen_products = product_date_hyphen(parsed_products_content_list)

2. With SLASH. Ex: 2022/04/14

In [20]:
def product_date_slash(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_date_slash_products = [datetime.today().strftime('%Y/%m/%d') for i in range(len(urls_products_list))]
    return final_date_slash_products

final_date_slash_products = product_date_slash(parsed_products_content_list)

3. Without symbols. Ex: 20220414

In [21]:
def product_date_number(parsed_products_content_list):

    def handling_error_vars_product(i,text):
        try:
            str(i).split(text)[1]
        except:
            return "none"
        else:
            return str(i).split(text)[1].split("';")[0].strip()
        
    final_date_number_products = [int(datetime.today().strftime('%Y%m%d')) for i in range(len(urls_products_list))]
    return final_date_number_products

final_date_number_products = product_date_number(parsed_products_content_list)

# B. p_wrangling

## Create the DATAFRAME

### a) Full DataFrame with a daily injection of the new scraped data

In [107]:
def conection_spreadsheet(scopes, credentials_location):
    scopes = scopes
    credentials = Credentials.from_service_account_file(
        credentials_location,
        scopes=scopes
    )
    gc = gspread.authorize(credentials)
    #si queremos alojar nuestras credenciales online tendríamos que hacer lo siguiente:
    # gc = gspread.service_account(filename='https://www.path/to/the/downloaded/file.json')
    print("conexion con spreadsheet realizada")
    return gc

gc = conection_spreadsheet(["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"], '../spreadsheets/json/credentials.json')

conexion con spreadsheet realizada


#### Step 1: Create or define the dataFrame

 - From a CSV if it exists
 - If not, from spreadsheets
 - If not, from this notebook
 - If not, we have to create a empty dataFrame

In [128]:
def main_df(location,gc):
#1. From a CSV if it exists
    try:
        df_single = pd.read_csv(location)
        print("df_single creado a partir de csv")
        return df_single
    
#2. If not, from spreadsheets    
    except: #si se ejecuta el except es que el try ha dado error, por lo que no existe en csv (pasamos a utilizar el backup de spreadsheets)   
        try:
            sheet = gc.open("business_afi_scraping_df_single").sheet1  #Abrir spreadhseet
            data_from_spreadsheets = sheet.get_all_records()  #Obtener todos los registros
            df_single = pd.DataFrame(data_from_spreadsheets)
            print("df_single creado a partir de spreadsheet")
            return df_single
        
#3. If not, from this notebook
        except: #si se ejecuta el except es que el try ha dado error, por lo que vamos a ver si ya estaba creado en el notebook       
            try:
                df_single.head()
                print("df_single creado a partir del notebook")
                return df_single
            
#4. If not, we have to create a empty dataFrame
            except: #si da error es que no existe en el notebook y lo crearemos desde cero
                columns = ["date_hyphen","date_slash","date_number","product_name","product_id","product_brand","price","status","url"]
                df_single = pd.DataFrame(columns=columns)
                print("df_single creado desde cero")
                return df_single
                #df_single["date_hyphen"] = df_single["date_hyphen"].astype("datetime64")
            else: #si no da error es que existe en el notebook y no haremos nada
                pass
            
        else: #si no da error es que existe en spreadsheets y no haremos nada
            pass
            
    else: #si no da error es que existe en csv y no haremos nada
        pass

df_single = main_df("../files/df_single.csv",gc)

df_single creado a partir de csv


#### Step 2: Append new records in the df_single dataFrame - pd.concat()

In [23]:
def df_append_new_files(final_date_hyphen_products,final_date_slash_products,final_date_number_products,final_name_products,final_id_products,final_brand_products,final_price_products_list,final_price_products_status,urls_products_list):
    df_append_new_files = pd.DataFrame({
        "date_hyphen":final_date_hyphen_products,
        "date_slash":final_date_slash_products,
        "date_number":final_date_number_products,
        "product_name":final_name_products,
        "product_id":final_id_products,
        "product_brand":final_brand_products,
        "price":final_price_products_list,
        "status":final_price_products_status,
        "url": urls_products_list
    })
    #df_append_new_files["date_hyphen"] = df_append_new_files["date_hyphen"].astype("datetime64")
    df_append_new_files["price"] = df_append_new_files["price"].astype("int64")
    
    return df_append_new_files

df_append_new_files = df_append_new_files(final_date_hyphen_products,final_date_slash_products,final_date_number_products,final_name_products,final_id_products,final_brand_products,final_price_products_list,final_price_products_status,urls_products_list)

In [24]:
def concat_df(df_single,df_append_new_files):
    if df_single.empty == True:
        df_single = pd.concat([df_single, df_append_new_files], ignore_index=True)
        return df_single
    elif df_append_new_files["date_hyphen"][0] == df_single["date_hyphen"][0]:
        pass
    elif df_append_new_files["date_hyphen"][0] != df_single["date_hyphen"][0]:
        df_single = pd.concat([df_single, df_append_new_files], ignore_index=True)
        return df_single
    
df_single = concat_df(df_single,df_append_new_files)

### b) Clean and Prepare the dataFrame

In [27]:
def corregir_gam_name(url,product_name):
    if url == "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/" and product_name == "none":
        return "intimate wm heat  Racing Silla Gamer"
    else:
        return product_name
        
def corregir_gam_id(url,product_id):
    if url == "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/" and product_id == "none":
        return "B075CK3GVJ"
    else:
        return product_id
        
def corregir_gam_brand(url,product_brand):
    if url == "https://sillasybienestar.com/gaming/sillas-gaming/review-individual/intimate-wm-heart-gaming/" and product_brand == "none":
        return "intimate wm heat"
    else:
        return product_brand

    
    
def corregir_gam_name_hbada_rep(url,product_name):
    if url == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/" and product_name == "none":
        return "Hbada B07V51M94R"
    else:
        return product_name
        
def corregir_gam_id_hbada_rep(url,product_id):
    if url == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/" and product_id == "none":
        return "B07V51M94R"
    else:
        return product_id
        
def corregir_gam_brand_hbada_rep(url,product_brand):
    if url == "https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/hbada-reposabrazos/" and product_brand == "none":
        return "Hbada"
    else:
        return product_brand

In [28]:
def correction_df_single(df_single):
    #intimate wm heat  Racing Silla Gamer
    df_single["product_name"] = df_single.apply(lambda df_single: corregir_gam_name(df_single["url"], df_single["product_name"]), axis=1)
    df_single["product_id"] = df_single.apply(lambda df_single: corregir_gam_id(df_single["url"], df_single["product_id"]), axis=1)
    df_single["product_brand"] = df_single.apply(lambda df_single: corregir_gam_brand(df_single["url"], df_single["product_brand"]), axis=1)

    #Hbada B07V51M94R
    df_single["product_name"] = df_single.apply(lambda df_single: corregir_gam_name_hbada_rep(df_single["url"], df_single["product_name"]), axis=1)
    df_single["product_id"] = df_single.apply(lambda df_single: corregir_gam_id_hbada_rep(df_single["url"], df_single["product_id"]), axis=1)
    df_single["product_brand"] = df_single.apply(lambda df_single: corregir_gam_brand_hbada_rep(df_single["url"], df_single["product_brand"]), axis=1)

    return df_single

df_single = correction_df_single(df_single)

### c) Create the price main of each product

In [29]:
def product_mean_dic(df_single):
    product_id_list_mean = df_single["product_id"].unique().tolist()
    mean_dic = {}
    for i in product_id_list_mean:
        mean_dic[i] = int(round(df_single[df_single["product_id"] == i]["price"].mean(),0))
    return mean_dic

product_mean = product_mean_dic(df_single### b) Clean and Prepare the dataFrame
)

In [30]:
def product_mean_dic_to_df(df_single,product_mean):
    df_single["product_mean"] = df_single["product_id"].apply(lambda x: product_mean[x])
    return df_single

df_single = product_mean_dic_to_df(df_single,product_mean)

In [31]:
def product_mean_status_func_values(product_price,product_mean):
    if product_price < product_mean and product_price > 0:
        return "precio por debajo de la media"
    elif product_price > product_mean and product_price > 0:
        return "precio por encima de la media"
    elif product_price == product_mean and product_price > 0:
        return "precio igual que la media"
    elif product_price == 0:
        return "producto descatalogado o sin stock"
    else:
        return "revisar, aviso"

In [32]:
def product_mean_status_func_apply(df_single):
    df_single["product_mean_status"] = df_single.apply(lambda x: product_mean_status_func_values(x["price"], x["product_mean"]), axis=1)
    return df_single

df_single = product_mean_status_func_apply(df_single)

### d) Create the category of each product

In [33]:
def categories_url(product_url):
    if "/ergonomia/sillas-ergonomicas/review-individual/" in product_url:
        return "ergonomia"
    elif "/oficina-y-escritorio/sillas-de-oficina/review-individual/" in product_url:
        return "oficina"
    elif "/ergonomia/sillas-de-rodillas/review-individual/" in product_url:
        return "rodilla"
    elif "/gaming/sillas-gaming/review-individual/" in product_url:
        return "gaming"
    else:
        return "alerta sin categoria"

In [34]:
def product_category(df_single):
    #df_single["product_category"] = df_single.apply(lambda x: categories_url(x["url"], axis=1))
    df_single["product_category"] = df_single["url"].apply(categories_url)
    return df_single

df_single = product_category(df_single)

## EXPORT the dataFrames to CSVs

In [35]:
def export_files(df_single, df_append_new_files):
    df_single_to_csv = df_single.to_csv("../files/df_single.csv", sep=",", index=False)
    df_append_new_files_to_csv = df_append_new_files.to_csv("../files/df_append_new_files_last_day.csv", sep=",", index=False)#+str(datetime.today().strftime('%Y-%m-%d'))+".csv", sep=",", index=False)
    out_of_stock_df = df_append_new_files[df_append_new_files["status"] != "correcto"]
    out_of_stock_df_to_csv = out_of_stock_df.to_csv("../files/out_of_stock_last_day.csv", sep=",", index=False)
    
    none_values = df_single[df_single["product_name"]=="none"].any().unique().tolist()
    if none_values == [True]:
        none_values_df = df_single[df_single["product_name"]=="none"]
        none_values_df_to_csv = none_values_df.to_csv("../files/non_values_last_day.csv", sep=",", index=False)
    
    return "files exported successfully"

export = export_files(df_single, df_append_new_files)
export

'files exported successfully'

In [121]:
df_single.head()

,date_hyphen,date_slash,date_number,product_name,product_id,product_brand,price,status,url,product_mean,product_mean_status,product_category
0,2022-04-14,2022/04/14,20220414,TLV TLV-MYX-801-1,B08T1TSMQQ,TLV,89,correcto,https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/tlv-myx-801-1/,89,precio igual que la media,ergonomia
1,2022-04-14,2022/04/14,20220414,Songmics OBN55BK,B07ZCJVFDJ,Songmics,0,sin_stock,https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/songmics-obn55bk/,98,producto descatalogado o sin stock,ergonomia
2,2022-04-14,2022/04/14,20220414,Cashoffice B081QDV9VR,B081QDV9VR,Cashoffice,83,correcto,https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cashoffice-silla-ergonomica/,83,precio igual que la media,ergonomia
3,2022-04-14,2022/04/14,20220414,Cedric B07SRYBJ6L,B07SRYBJ6L,Cedric,252,correcto,https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/cedric/,252,precio igual que la media,ergonomia
4,2022-04-14,2022/04/14,20220414,Noblewell B08DR42HFV,B08DR42HFV,Noblewell,139,correcto,https://sillasybienestar.com/ergonomia/sillas-ergonomicas/review-individual/noblewell/,139,precio igual que la media,ergonomia


In [122]:
df_single.tail(5)

,date_hyphen,date_slash,date_number,product_name,product_id,product_brand,price,status,url,product_mean,product_mean_status,product_category
363,2022-04-21,2022/04/21,20220421,diablo X-horn,B07Z9CTB5C,diablo,274,correcto,https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-horn/,259,precio por encima de la media,gaming
364,2022-04-21,2022/04/21,20220421,diablo X-Ray,B07Q57WD7S,diablo,279,correcto,https://sillasybienestar.com/gaming/sillas-gaming/review-individual/diablo-x-ray/,264,precio por encima de la media,gaming
365,2022-04-21,2022/04/21,20220421,DXRacer Formula F08,B073GF2L1H,dxracer,349,correcto,https://sillasybienestar.com/gaming/sillas-gaming/review-individual/dxracer-formula-f08/,355,precio por debajo de la media,gaming
366,2022-04-21,2022/04/21,20220421,Newskill Kitsune,B073TWN367,Newskill,169,correcto,https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-kitsune/,169,precio igual que la media,gaming
367,2022-04-21,2022/04/21,20220421,Newskill Takamikura,B019HA9MKM,Newskill,189,correcto,https://sillasybienestar.com/gaming/sillas-gaming/review-individual/newskill-takamikura/,189,precio igual que la media,gaming


In [123]:
len(df_single)

368

# C. p_analysis

We are going to study the values looking for errors to create alerts via email

### - EMAIL ALERTS

In [38]:
#funcion para enviar emails
def business_afi_email_sender(receiver_email, filename_location, csv_name, body_email):

    subject = "sillas y bienestar | "+ csv_name + " | " + str(datetime.today().strftime('%Y-%m-%d'))
    body = body_email #"CSV como backup con el dataframe diario en formato csv"
    sender_email = "businessafiliacion@gmail.com"
    receiver_email = receiver_email
    password = "kwjyhhdbgnemzhhp"
    
    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email  # Recommended for mass emails
    
    # Add body to email
    message.attach(MIMEText(body, "plain"))
    
    filename = filename_location
    
    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    
    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)
    
    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )     
    
    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
        
    return "email " + csv_name + " sended"

#### 1) out of stock or discontinued

In [39]:
#out of stock o descatalogado
business_afi_email_sender("businessafiliacion@gmail.com","../files/out_of_stock_last_day.csv", "out_of_stock_last_day.csv", "CSV con productos descatalogados o fuera de stock")

'email out_of_stock_last_day.csv sended'

#### 2) none values

In [40]:
def email_none_values(df_single, df_append_new_files):
    none_values = df_single[df_single["product_name"]=="none"].any().unique().tolist()
    if none_values == [True]:       
        business_afi_email_sender("businessafiliacion@gmail.com", "../files/non_values_last_day.csv", "non_values_last_day.csv", "CSV con valores none")       
        return "ALERT ALVARO, exists none values"
    else:
        return "does not exist none values"

In [41]:
email_none_values(df_single, df_append_new_files)

'does not exist none values'

#### 3) send the csv to have a backup

In [42]:
business_afi_email_sender("businessafiliacion@gmail.com", "../files/df_single.csv", "df_single.csv", "CSV PRINCIPAL como backup con el dataframe diario con todos los datos en formato csv")

'email df_single.csv sended'

In [43]:
business_afi_email_sender("businessafiliacion@gmail.com", "../files/df_append_new_files_last_day.csv", "df_append_new_files_last_day.csv", "CSV CON DATOS DEL DIA como backup con el dataframe diario en formato csv")

'email df_append_new_files_last_day.csv sended'

# D. p_reporting

We are going to send the different DataFrames to spreadsheet

#### INDEX LIBRARY
https://docs.gspread.org/en/latest/

#### USER GUIDE
https://docs.gspread.org/en/latest/user-guide.html

The conection has been done in the "p_wrangling" module. The output is located in the variable "gc"

In [129]:
def update_spreadsheet(gc, spreadsheet_name, worksheet_name, dataframe):
    #Open the spreadhseet
    sheet = gc.open(spreadsheet_name).worksheet(worksheet_name)
    
    #Clear and Update the Worksheet
    sheet.clear()
    sheet.update('A1:L1',[dataframe.columns.tolist()])
    sheet.update('A2:L' + str(len(dataframe)+1), dataframe.values.tolist())
    
    return "worksheet updated"

In [97]:
#sheet1 - df_single
update_spreadsheet(gc, "business_afi_scraping_df_single", "df_single", df_single)

'worksheet actualizada'

In [98]:
#sheet2 - df_append_new_files
update_spreadsheet(gc, "business_afi_scraping_last_day_files", "df_append_new_files", df_append_new_files)

'worksheet actualizada'

In [105]:
#sheet3 - out_of_stock_df
def out_of_stock_spreadsheet(df_append_new_files):
    out_of_stock_df = df_append_new_files[df_append_new_files["status"] != "correcto"]
    update_spreadsheet(gc, "business_afi_scraping_last_day_files", "out_of_stock_df", out_of_stock_df)
    
    return "worksheet actualizada"
out_of_stock_spreadsheet(df_append_new_files)

'worksheet actualizada'

In [103]:
#sheet4 - none_values_df
def non_values_spreadsheet(df_single):
    none_values = df_single[df_single["product_name"]=="none"].any().unique().tolist()
    if none_values == [True]:
        none_values_df = df_single[df_single["product_name"]=="none"]
        update_spreadsheet(gc, "business_afi_scraping_last_day_files", "none_values_df", none_values_df)
        
        return "worksheet actualizada"
    else:
        return "worksheet actualizada, pero no hay valores none"
    
non_values_spreadsheet(df_single)

'worksheet actualizada, pero no hay valores none'